pip install git+https://github.com/asimov-academy/Asimov_TV_connector.git

https://python.plainenglish.io/extracting-data-from-trading-view-253919ba7194

In [42]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = open('login_tv.txt', 'r').read()
password = open('passwd_tv.txt', 'r').read()

tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [43]:
# # tv.search_symbol('PETR4','BMFBOVESPA')
# df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
# df.head(3)

In [44]:
df = tv.get_hist(symbol='CAML3',exchange='BMFBOVESPA',interval=Interval.in_1_minute, n_bars= 1)
df.tail(50)

,symbol,open,high,low,close,volume
datetime,,,,,,
2025-12-01 11:20:00,BMFBOVESPA:CAML3,5.79,5.79,5.79,5.79,100.0


In [45]:
acoes = [
    "UNIP6",
    "BRAP4",
    "POMO4",
    "LOGG3",
    "CMIG4",
    "ROMI3",
    "FESA4",
    "RANI3",
    "PETR4",
    "TRIS3"
]


In [50]:
nbars=5
for idx,row in enumerate(acoes):
    i=row
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= nbars)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'}
                       ,inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= nbars)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'}
                       ,inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.tail()

,UNIP6_open,UNIP6_high,UNIP6_low,UNIP6_close,UNIP6_volume,BRAP4_open,BRAP4_high,BRAP4_low,BRAP4_close,BRAP4_volume,...,PETR4_open,PETR4_high,PETR4_low,PETR4_close,PETR4_volume,TRIS3_open,TRIS3_high,TRIS3_low,TRIS3_close,TRIS3_volume
datetime,,,,,,,,,,,,,,,,,,,,,
2025-11-25 10:00:00,60.50,61.25,58.88,58.94,224900.0,18.50,18.72,18.41,18.66,2720100.0,...,32.35,32.44,32.09,32.28,28431100.0,8.46,8.64,8.40,8.63,378200.0
2025-11-26 10:00:00,58.80,60.24,58.80,59.45,196100.0,18.71,19.16,18.71,19.15,3418300.0,...,32.38,32.41,32.10,32.23,25042100.0,8.63,8.88,8.58,8.81,157900.0
2025-11-27 10:00:00,59.45,61.82,59.45,60.09,275500.0,19.15,19.19,19.00,19.06,2100700.0,...,32.28,32.40,32.20,32.40,16087400.0,8.80,8.85,8.67,8.78,202200.0
2025-11-28 10:00:00,60.15,62.54,60.15,60.49,397400.0,19.20,19.56,19.11,19.43,3501400.0,...,32.27,32.31,31.29,31.79,60395900.0,8.70,8.90,8.70,8.89,261100.0
2025-12-01 10:00:00,60.51,61.32,59.75,59.97,78700.0,19.51,19.51,19.36,19.40,358200.0,...,31.95,32.14,31.57,32.11,10175300.0,8.89,8.92,8.75,8.89,53700.0


In [51]:

# # 1. Transpor preservando índice
# df2 = df.reset_index().T

# # 2. Garantir comportamento futuro: inferir tipos
# df2.index = df2.index.infer_objects()
# df2.columns = df2.iloc[0]
# df2 = df2[1:]

# 3. Função segura para transformar decimais em vírgula
def convert_decimal(x):
    try:
        f = float(x)
        return str(f).replace('.', ',')
    except:
        return x

# 4. Usar .map() em cada coluna (substitui applymap sem warnings)
df2 = df.map(convert_decimal)

# 5. Salvar
df2.to_csv("data/position.csv", sep=';', index=True)

In [5]:
url0 = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/'

url  = {'ibra50' :url0+'Cart_IBr50.csv',
        'ibra100':url0+'Cart_IBr100.csv',
        'idiv'   :url0+'Cart_Idiv.csv',
        'ibov'   :url0+'Cart_Ibov.csv',
        'small'  :url0+'Cart_Small.csv'
}
indice = pd.read_csv(url['ibov'] ,
                     sep=';' , decimal=',' , thousands='.' ,
                     skiprows=1 , skipfooter=2 , encoding='latin-1',
                     index_col=False , engine='python')

indice

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,ALOS3,ALLOS,ON NM,476976044,0.500
1,ABEV3,AMBEV S/A,ON,4394835131,2.724
2,ASAI3,ASSAI,ON NM,1345897506,0.636
3,AURE3,AUREN,ON NM,323738747,0.145
4,AZZA3,AZZAS 2154,ON NM,136643320,0.257
...,...,...,...,...,...
79,VAMO3,VAMOS,ON NM,485166826,0.088
80,VBBR3,VIBRA,ON NM,1113939036,1.154
81,VIVA3,VIVARA S.A.,ON NM,123160591,0.154
82,WEGE3,WEG,ON NM,1482105837,2.922


In [6]:
lista = ['PETR3','PETR4','VALE3','BBAS3']

In [7]:
for idx,row in enumerate(lista):
    i=row
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 100)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [8]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]